In [181]:
#importar librerias a usar
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [182]:
#especificar formato de campo fecha para carga directa desde lectura de pandas
#req n°2 De haber fechas, deberán tener el formato AAAA-mm-dd
custom_date_parser = lambda x: pd.to_datetime(x, format= "%B %d, %Y", errors='coerce')
#importar csv solicitados 
amazon = pd.read_csv("D://Henry//PI 01//PI01-Data-Engineering//Datasets//amazon_prime_titles-score.csv",parse_dates=['date_added'], date_parser=custom_date_parser)
disney_plus = pd.read_csv("D://Henry//PI 01//PI01-Data-Engineering//Datasets//disney_plus_titles-score.csv",parse_dates=['date_added'], date_parser=custom_date_parser)
hulu = pd.read_csv("D://Henry//PI 01//PI01-Data-Engineering//Datasets//hulu_titles-score (2).csv",parse_dates=['date_added'], date_parser=custom_date_parser)
netflix = pd.read_csv("D://Henry//PI 01//PI01-Data-Engineering//Datasets//netflix_titles-score.csv",parse_dates=['date_added'], date_parser=custom_date_parser)

In [183]:
#Transformaciones de datos
#req n°1 Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)
amazon["show_id"] = "a" + amazon["show_id"]
disney_plus["show_id"] = "d" + disney_plus["show_id"]
hulu["show_id"] = "h" + hulu["show_id"]
netflix["show_id"] = "n" + netflix["show_id"]
movie_db = pd.concat([amazon,disney_plus,hulu,netflix]).reset_index(drop=True)


In [184]:
#req n°X1 encontré carga de datos en columnas erradas, así que las reubiqué donde deberian
modificaciones_1 = movie_db[movie_db['rating'].str.contains('min', na=False)].index.to_list()
modificaciones_2 = movie_db[movie_db['rating'].str.contains('Season', na=False)].index.to_list()

modificaciones = modificaciones_1 + modificaciones_2

for i in modificaciones:
    movie_db.loc[i,"duration"] = movie_db.loc[i,"rating"]
    movie_db.loc[i,"rating"] = np.nan


In [185]:
#req n°3 Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”
movie_db["rating"] = movie_db["rating"].replace(np.nan,"G")


In [186]:
#req n°4 Los campos de texto deberán estar en minúsculas, sin excepciones
movie_db = movie_db.applymap(lambda x: x.lower() if isinstance(x,str) else x)

In [187]:
#req n°5 El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)
dic = {'min':'min', np.nan:np.nan, 'season':'season', 'seasons':'season'}

movie_db[["duration_int","duration_type"]]= movie_db["duration"].apply(lambda x: pd.Series(str(x).split(" ")))
movie_db["duration_int"] = movie_db["duration_int"].apply(lambda x: np.nan if x == "nan" else x)
movie_db["duration_int"] = movie_db["duration_int"].astype('Int64')
movie_db["duration_type"] = movie_db["duration_type"].map(dic)

In [188]:
#exportamos el csv limpio
movie_db.to_csv("movie_db.csv",encoding="UTF8")

In [189]:
#1. Cantidad de veces que aparece una keyword en el título de peliculas/series, por plataforma
## pregunta: si en 1 mismo titulo, aparece la palabra 2 veces, la contamos 2 veces o solo 1 por titulo?
def get_word_count(plataforma,keyword):
    dicc = {"netflix":"n","amazon":"a","hulu":"h","disney_plus":"d"}

    df_temp = movie_db[(movie_db.show_id.str[0] == dicc[plataforma])&(movie_db['title'].str.contains(keyword, na=False))]
    data = df_temp.groupby(df_temp.show_id.str[0]).count().iloc[:,0]
    data = data.rename_axis("Cantidad").rename({v: k for k, v in dicc.items()})
    return data.to_dict()

get_word_count("netflix","love")

{'netflix': 196}

In [190]:
#2. Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año
def get_score_count(plataforma,puntaje,anio):
    dicc = {"netflix":"n","amazon":"a","hulu":"h","disney_plus":"d"}

    df_temp = movie_db[(movie_db["score"]>puntaje)&(movie_db["release_year"]==anio)&(movie_db.show_id.str[0] == dicc[plataforma])&(movie_db["type"] == "movie")]
    data = df_temp.groupby(df_temp.show_id.str[0]).count().iloc[:,0]
    data = data.rename({v: k for k, v in dicc.items()})
    return data.to_dict()

get_score_count("netflix",85,2010)

{'netflix': 20}

In [191]:
#3. La segunda película con mayor score para una plataforma determinada, según el orden alfabético de los títulos.
#df.sort_values(['a', 'b'], ascending=[True, False])
def get_second_score(plataforma):
    dicc = {"netflix":"n","amazon":"a","hulu":"h","disney_plus":"d"}

    df_temp = movie_db[(movie_db.show_id.str[0] == dicc[plataforma])&(movie_db.type == "movie")].sort_values(["score","title"],ascending=[False,True]).reset_index(drop=True)
    df_temp = df_temp[["title","score"]].iloc[1,:]
    return df_temp.to_dict()
    
get_second_score("amazon")

{'title': 'abilene town', 'score': 100}

In [192]:
#4. Película que más duró según año, plataforma y tipo de duración

def get_longest(plataforma,tipo_duracion,anio):
    dicc = {"netflix":"n","amazon":"a","hulu":"h","disney_plus":"d"}

    temp = movie_db[(movie_db.show_id.str[0] == dicc[plataforma])&(movie_db.release_year == anio)&(movie_db.duration_type == tipo_duracion)&(movie_db.type == "movie")].sort_values(["duration_int"],ascending=False).reset_index(drop=True)[["title","duration_int","duration_type"]].iloc[0,:]
    return temp.to_dict()

get_longest('netflix','min',2016)

{'title': 'sairat', 'duration_int': 173, 'duration_type': 'min'}

In [193]:
#5. Cantidad de series y películas por rating
def get_rating_count(rating_cat):
    return len(movie_db[movie_db["rating"]==rating_cat])

get_rating_count('18+')

1243

In [194]:
import requests
n1 = requests.get("https://1r78k3.deta.dev/get_word_count/netflix/love")
n2 = requests.get("https://1r78k3.deta.dev/get_score_count/netflix/85/2010")
n3 = requests.get("https://1r78k3.deta.dev/get_second_score/amazon")
n4 = requests.get("https://1r78k3.deta.dev/get_longest/netflix/min/2016")
n5 = requests.get("https://1r78k3.deta.dev/get_rating_count/18+")

In [195]:
n1.json(),n2.json(),n3.json(),n4.json(),n5.json()

({'netflix': 196},
 {'netflix': 20},
 {'title': 'abilene town', 'score': 100},
 {'title': 'sairat', 'duration_int': 173.0, 'duration_type': 'min'},
 1243)